<img src="./img/oscon.png" width="400">

# Session 4: TensorBoard Visualization

In this session, we explore the use of TensorBoard, to visualize the training + computation graph of the simple softmax classifier trained on MNIST we saw in previous sessions. 

## Reminder: MNIST softmax classifier

In [1]:
from __future__ import print_function

import tensorflow as tf
import sys,os
from tensorflow.examples.tutorials.mnist import input_data
DATA_DIR = os.path.join(os.environ["HOME"], "data") if not 'win32' in sys.platform else "c:\\tmp\\data"

NUM_STEPS = 1000
MINIBATCH_SIZE = 100

In [2]:
data = input_data.read_data_sets(DATA_DIR, one_hot=True)

Extracting c:\tmp\data\train-images-idx3-ubyte.gz
Extracting c:\tmp\data\train-labels-idx1-ubyte.gz
Extracting c:\tmp\data\t10k-images-idx3-ubyte.gz
Extracting c:\tmp\data\t10k-labels-idx1-ubyte.gz


In [3]:
x = tf.placeholder(tf.float32, [None, 784])
W = tf.Variable(tf.zeros([784, 10]))
b = tf.Variable(tf.zeros([10]))

y_true = tf.placeholder(tf.float32, [None, 10])
y_pred = tf.add(tf.matmul(x, W),b)

In [4]:
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = y_pred, labels = y_true))

gd_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)

correct_mask = tf.equal(tf.argmax(y_pred, 1), tf.argmax(y_true, 1))
accuracy = tf.reduce_mean(tf.cast(correct_mask, tf.float32))

In [5]:
sess = tf.InteractiveSession()

In [6]:
# Train
sess.run(tf.global_variables_initializer())

for _ in range(NUM_STEPS):
    batch_xs, batch_ys = data.train.next_batch(MINIBATCH_SIZE)
    sess.run(gd_step, feed_dict={x: batch_xs, y_true: batch_ys})


In [7]:
# Test
ans = sess.run(accuracy, feed_dict={x: data.test.images, y_true: data.test.labels})

print ("Accuracy: {:.4}%".format(ans*100))
sess.close()

Accuracy: 91.74%


## Adding summaries

We'd like to visualize our TensorFlow graph, and plot metrics from the training process. For graph visualization, we add variable name scopes to define a hierarchy on nodes in the graph that is used by TensorBoard when displaying the graph. Summary operations are attached to nodes in the graph we are interested in plotting. 

In [8]:
tf.reset_default_graph()
sess = tf.InteractiveSession()
with tf.name_scope('input_x'):
    x = tf.placeholder(tf.float32, [None, 784])
with tf.name_scope('input_label'):
    y_true = tf.placeholder(tf.float32, [None, 10]) 
with tf.name_scope('input_reshape'):
    image_shaped_input = tf.reshape(x, [-1, 28, 28, 1])
    tf.summary.image('input', image_shaped_input, 10)

In [9]:
with tf.name_scope('weights'):    
    W = tf.Variable(tf.zeros([784, 10]))
    with tf.name_scope('summaries'):
        mean = tf.reduce_mean(W)
        tf.summary.scalar('mean', mean)
        tf.summary.histogram('histogram', W)
with tf.name_scope('biases'):    
    b = tf.Variable(tf.zeros([10]))
    with tf.name_scope('summaries'):
        mean = tf.reduce_mean(b)
        tf.summary.scalar('mean', mean)
        tf.summary.histogram('histogram', b)

with tf.name_scope('Wx_b'):     
    y_pred = tf.add(tf.matmul(x, W),b)
    tf.summary.histogram('Wx_b', y_pred)

In [10]:
with tf.name_scope('loss'): 
    cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = y_pred, labels = y_true))
    tf.summary.scalar('loss', cross_entropy)
    
with tf.name_scope('train'):
    gd_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)
with tf.name_scope('correct_pred'):
    correct_mask = tf.equal(tf.argmax(y_pred, 1), tf.argmax(y_true, 1))
with tf.name_scope('accuracy'):
    accuracy = tf.reduce_mean(tf.cast(correct_mask, tf.float32))


We next merge all the summaries into one op, generating a serialized object with all summary data at a given step. This serialized object is passed to a FileWriter, which writes the data to disk. FileWriter takes a logdir, which is the path to the folder we save all data. If given a TensorFlow graph object, FileWriter will also create a very useful visualization of our computation graph.

In [11]:
# Merge all the summaries and write them to disk
merged = tf.summary.merge_all()
train_writer = tf.summary.FileWriter(os.path.join(DATA_DIR,"logs\\demo\\train"), sess.graph)
test_writer = tf.summary.FileWriter(os.path.join(DATA_DIR,"logs\\demo\\test"))

In [12]:
# Train
sess.run(tf.global_variables_initializer())

for i in range(NUM_STEPS):

    batch_xs, batch_ys = data.train.next_batch(MINIBATCH_SIZE)
    summary, _ = sess.run([merged, gd_step], feed_dict={x: batch_xs, y_true: batch_ys})
    train_writer.add_summary(summary, i)
    if i % 10 == 0:  # Record summaries and test-set accuracy
        summary, acc = sess.run([merged, accuracy], feed_dict={x: data.test.images, y_true: data.test.labels})
        test_writer.add_summary(summary, i)
        print('Accuracy at step %s: %s' % (i, acc))
train_writer.close()
test_writer.close()        

Accuracy at step 0: 0.1652
Accuracy at step 10: 0.7661
Accuracy at step 20: 0.8238
Accuracy at step 30: 0.8661
Accuracy at step 40: 0.8638
Accuracy at step 50: 0.8836
Accuracy at step 60: 0.8758
Accuracy at step 70: 0.8847
Accuracy at step 80: 0.8886
Accuracy at step 90: 0.8875
Accuracy at step 100: 0.8866
Accuracy at step 110: 0.8959
Accuracy at step 120: 0.9
Accuracy at step 130: 0.9003
Accuracy at step 140: 0.8925
Accuracy at step 150: 0.9053
Accuracy at step 160: 0.9043
Accuracy at step 170: 0.9023
Accuracy at step 180: 0.8982
Accuracy at step 190: 0.9026
Accuracy at step 200: 0.9046
Accuracy at step 210: 0.9011
Accuracy at step 220: 0.9
Accuracy at step 230: 0.9075
Accuracy at step 240: 0.9035
Accuracy at step 250: 0.9095
Accuracy at step 260: 0.904
Accuracy at step 270: 0.9073
Accuracy at step 280: 0.9082
Accuracy at step 290: 0.9119
Accuracy at step 300: 0.9138
Accuracy at step 310: 0.9027
Accuracy at step 320: 0.9109
Accuracy at step 330: 0.9081
Accuracy at step 340: 0.9072
Acc

In [13]:
print(DATA_DIR)

c:\tmp\data


Run the command line: tensorboard --logdir=demo:DATA_DIR [replace DATA_DIR with your path]

Then, in your browser, go to localhost:6006

Note: TensorBoard recursively walks the directory tree rooted at logdir looking for subdirectories that contain tfevents data. 

## Quick hands-on

Take TensorBoard for a spin:

* Add some more types of summaries (you'll need to use the suitable TensorFlow ops that you saw earlier!):
    * Max
    * Min
    * Standard deviation
    
* Run and check out results. To avoid overwriting of existing data, add another subdirectory to the log directory given to FileWriter. For example, use tf.summary.FileWriter(os.path.join(DATA_DIR,"logs\\ex1\\train"), sess.graph).


In [ ]:
# %load ..\solutions\tfboard_ex1.py    